# Ollama Introduction
This little notebook is a brief introduction to Ollama, a tool for interacting with open LLMs deployed on Ollama server (the server can also run locally).

In [6]:
from ollama import Client
import ollama

# initialize the Ollama client with the specified host
# ollama_host = "http://10.167.31.201:11434/"
ollama_host = "http://localhost:11434/"
client = Client(host=ollama_host)

In [12]:
# Get a list of all models that are currently downloaded
models = client.list()
print(models.models)

# MODEL_NAME = "gemma3:27b"
MODEL_NAME = "mistral:7b"

# mistral-small3.2:24b

[Model(model='mistral:7b', modified_at=datetime.datetime(2025, 6, 25, 14, 20, 0, 950236, tzinfo=TzInfo(+02:00)), digest='3944fe81ec14610e0852c3d915768ee8d507ea541387fdfcbbf9edaa0c757734', size=4113301822, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='7.2B', quantization_level='Q4_0')), Model(model='gemma3:latest', modified_at=datetime.datetime(2025, 6, 11, 16, 51, 6, 220107, tzinfo=TzInfo(+02:00)), digest='a2af6cc3eb7fa8be8504abaf9b04e88f17a119ec3f04a3addf55f92841195f5a', size=3338801804, details=ModelDetails(parent_model='', format='gguf', family='gemma3', families=['gemma3'], parameter_size='4.3B', quantization_level='Q4_K_M')), Model(model='gemma3:27b', modified_at=datetime.datetime(2025, 6, 11, 16, 45, 59, 91280, tzinfo=TzInfo(+02:00)), digest='a418f5838eaf7fe2cfe0a3046c8384b68ba43a4435542c942f9db00a5f342203', size=17396936941, details=ModelDetails(parent_model='', format='gguf', family='gemma3', families=['gemma3'], parame

# Question extraction

In [13]:
import json
import os
from pydantic import BaseModel
from typing import List
import re
import unicodedata


def clean_text_for_model(text):
    # 1. Remove Unicode replacement characters (�)
    text = text.replace('\uFFFD', '')
    
    # 2. Normalize Unicode (removes weird byte leftovers)
    text = unicodedata.normalize('NFKC', text)
    
    # 3. Remove non-ASCII characters, but keep newlines
    text = re.sub(r'[^\x20-\x7E\n]+', '', text)
    
    # 4. Collapse excessive whitespace
    text = re.sub(r'[ \t]{2,}', ' ', text)  # multiple spaces/tabs → one space
    text = re.sub(r'\n{3,}', '\n\n', text)  # 3+ newlines → 2

    return text.strip()


class QuestionResponse(BaseModel):
    questions: List[str]

def get_questions_from_review(review: str) -> List[str]:
    """
    Extracts scientific questions from a peer review.
    """
    response = client.chat(
        model=MODEL_NAME,
        messages=[
            {"role": "user",
            "content": (
                "You are given a peer review of a scientific article.\n\n"
                "Your task is to extract all **scientific, information-seeking questions** that the reviewer asked. It might contain no valid question, return empty list if that's the case. \n\n"

                "These questions should:\n"
                "- Be explicitly posed by the reviewer, not implied or inferred from a statement. Must be a question.\n"
                "- Seek clarification or justification about methods, data, results etc \n"
                "- Be grounded in the article’s scientific content\n"
                "- Be answerable by the authors with more scientific explanation or evidence\n\n"

                "**DO NOT include** questions or comments that are:\n"
                "- Editorial such as grammar, spelling, formatting, or structure\n"
                "- Rhetorical, evaluative, or based on the reviewer’s opinions \n"
                "- Referring to figures(e.g., “Figure 2, Figs. 3”)\n\n"

                "Rephrase valid questions to be self-contained and precise. Each question should focus on a single scientific subject. Add any missing context that is necessary to make the question understandable on its own. \n\n"

                "### Example 1:\n"
                "Input: \"In Table 2, the authors claim the signal-to-noise ratio improved significantly , but they don’t explain how it was measured. Was this ratio calculated over multiple trials or just a single run?\"\n"
                "Output: How was the signal-to-noise ratio measured in the modified setup—over multiple trials or a single run?\n\n"
                "### Example 2:\n"
                "Input: \"In line 32-35, why is f(\\nu_1) being biquadrate exponential distribution function, etc\"\n"
                "Output: What is the reasoning behind choosing a biquadrate exponential distribution function for f(ν₁)?\n\n"
                "### Example 3:\n"
                "Input: \"It is important to demonstrate this phenomenon occuring in vivo using primary T cells. The manuscript over interprets some of the data. \"\n"
                "Output: [] (No scientific, information-seeking question found.) \n\n"
                "### Example 4:\n"
                "Input: \"L300-309.Please be clear about how it links to the other material. I also recommend adding a column detailing the quantum gate counts used in the ADAPT-VQE. \"\n"
                "Output: [] (No scientific, information-seeking question found.) \n\n"
                "### Example 5:\n"
                "Input: \"The authors state that the results are statistically significant, but they do not provide the p-values. What are them?\"\n"
                "Output: What are the p-values for the key results presented in the study?\n\n"
                
                f"Now apply the same process to the following review:\n\n{review}"
            )
            }
        ],
        options={
            "temperature": 0.4,
            "num_predict": 1024,
        },
        format=QuestionResponse.model_json_schema()
    )

    structured_response = QuestionResponse.model_validate_json(response.message.content)
    return structured_response.questions

def filter_questions_regex(questions: List[str]) -> List[str]:
    """
    Filters out any questions with figure, line and table reference.
    """
    pattern = re.compile(
        r'('
        r'\b(?i:'
            r'\bfig(?:ure)?(?:s)?(?:\.)?(?:\d+)?\b'  # fig, figs, fig., figs., figure, figures + number, case-insensitive
            r'|' 
            r'lines?\s*\d+(?:[-–]\d+)?'             # line + number or number range (e.g., line 5 or line 5-10), case-insensitive
            r'|'
            r'table\s*\d+'                          # table + number, case-insensitive
        r')'
        r'|'  
        r'\bL\d+(?:-\d+)?'                          # Capital letter L + number or number range (e.g., L5 or L5-10), case-sensitive
        r')\b'
    )

    filtered_questions = [q for q in questions if not pattern.search(q)] 
    return filtered_questions


In [ ]:


base_dir = 'test-set'

for dir in os.listdir(base_dir):
    dir_path = os.path.join(base_dir, dir)
    if os.path.isdir(dir_path):
        print(f"Processing directory: {dir_path}")

        questions_json = os.path.join(dir_path, 'questions.json')
        # if os.path.exists(questions_json):
        #     print(f"Questions file already exists: {questions_json}")
        #     continue

        # Load the peer reviews from a JSON file
        review_file = os.path.join(dir_path, 'all_reviews.json')

        with open(review_file, 'r', encoding='utf-8') as file:
            reviews = json.load(file)
        
        result = []
        # Process each review
        for review in reviews:
            review = clean_text_for_model(review)
            if not review.strip():
                print("Empty review found, skipping.")
                continue
            try:
                questions = get_questions_from_review(review)
            except Exception as e:
                print(f"Error processing review in {review_file}: {e}")
                # try again
                try:
                    questions = get_questions_from_review(review)
                except Exception as e:
                    print(f"Error processing review again in {review_file}: {e}")
                    questions = []
            result.extend(questions)

        if not result:
            print(f"No questions extracted from {review_file}")
            continue

        result = list(set(result)) # Ensure uniqueness

        # Filter the questions
        filtered_questions = filter_questions_regex(result)
        if filtered_questions is None:
            print(f"No questions left after filtering for {review_file}")
            continue

        print(filtered_questions)

        # with open(questions_json, 'w', encoding='utf-8') as file:
        #     json.dump(filtered_questions, file, ensure_ascii=False, indent=4)
        # print(f"Questions saved to {questions_json}")

Processing directory: test-set\1-10


# Answer extraction


In [71]:
from pydantic import BaseModel
from typing import Dict
import re
import unicodedata


def clean_text_for_model(text):
    # 1. Remove Unicode replacement characters (�)
    text = text.replace('\uFFFD', '')
    
    # 2. Normalize Unicode (removes weird byte leftovers)
    text = unicodedata.normalize('NFKC', text)
    
    # 3. Remove non-ASCII characters, but keep newlines
    text = re.sub(r'[^\x20-\x7E\n]+', '', text)
    
    # 4. Collapse excessive whitespace
    text = re.sub(r'\n', ' ', text)  # newlines → space
    text = re.sub(r'[ \t]{2,}', ' ', text)  # multiple spaces/tabs → one space

    return text.strip()

class AnswerResponse(BaseModel):
    QAPairs: Dict[str, str]

def find_answer_in_rebuttal(question, rebuttal_text):
    """
    Finds the answer to a given question within a rebuttal text.
    If the rebuttal does not answer the question, return an empty string.
    """
    response = client.chat(
        model=MODEL_NAME,
        messages=[
            {
                "role": "user",
                "content": (
                    f"You are given a scientific question: {question}.\n\n"
                    f"Your task is to find an answer to the given question from the following rebuttal. The rebuttal might contains both reviewer comments and author replies.\n\n"
                    "Return only the answer in plain text. You may rephrase for better understanding. If no corresponding response is found, return 'none'. Do not repeat the question. Do NOT summarize the rebuttal.\n"

                    f"Rebuttal text:\n{rebuttal_text}\n\n"
                    "Answer:"
                )

            }
        ],
        options={
            "temperature": 0.2,
            "num_predict": 512,
        }
    )
    answer = response.message.content.strip()
    # Clean the answer
    if answer.lower() in ["", "none", "n/a"] or any(
        phrase in answer.lower() for phrase in ["the rebuttal does not contain", "no answer found"]
    ):
        answer = None

    return answer


In [72]:
import spacy
from textwrap import wrap
import json
import os

# spacy.cli.download("en_core_web_md")
nlp = spacy.load("en_core_web_md") 

MAX_CHARS = 2000

def chunk_by_sentence_count(text, max_characters=MAX_CHARS):
    doc = nlp(text)
    chunks = []
    current_chunk = ""

    for sent in doc.sents:
        sentence_text = sent.text.strip()
        if current_chunk:
            current_chunk += " " + sentence_text
        else:
            current_chunk = sentence_text

        if len(current_chunk) >= max_characters:
            chunks.append(current_chunk.strip())
            current_chunk = ""

    # Add remaining chunk
    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

def extract_answer(paper_dir):
    questions_json = os.path.join(paper_dir, 'questions.json')
    with open(questions_json, 'r', encoding='utf-8') as file:
        questions = json.load(file)

    final_qa = {}

    rebuttal_json = os.path.join(paper_dir, 'all_rebuttals.json')
    with open(rebuttal_json, "r", encoding="utf-8") as f:
        rebuttals = json.load(f)

    rebuttals_list = []
    for r in rebuttals:
        text = clean_text_for_model(r)
        chunks = chunk_by_sentence_count(text)
        rebuttals_list.append(chunks)
        print(f"Rebuttal split into {len(chunks)} chunks.")

        
    for question in questions:
        print(f"Finding answer for question: {question}")
        
        for rebuttal in rebuttals_list:
            found = False
            for chunk in rebuttal:
                answer = find_answer_in_rebuttal(question, chunk)
                if answer:
                    print(f"Answer found in chunk: {answer}")
                    final_qa[question] = answer
                    found = True
                    break  # Stop searching once we find an answer
                print(f"No answer found in chunk, moving to next chunk...")
            if found:
                break
                
    # Save the final Q&A pairs to a JSON file
    final_qa_json = os.path.join(paper_dir, 'final_qa.json')
    with open(final_qa_json, 'w', encoding='utf-8') as file:
        json.dump(final_qa, file, ensure_ascii=False, indent=4)



In [73]:
base_dir = 'test-set'

for dir in os.listdir(base_dir):
    paper_dir = os.path.join(base_dir, dir)
    if os.path.isdir(paper_dir):
        print(f"Processing directory: {paper_dir}")
        questions_json = os.path.join(paper_dir, 'questions.json')
        rebutttal_json = os.path.join(paper_dir, 'all_rebuttals.json')
        if not os.path.exists(questions_json) or not os.path.exists(rebutttal_json):
            print(f"Questions file or rebuttal file does not exist, skipping{paper_dir}.")
            continue
        # Extract answers for the questions in the paper
        extract_answer(paper_dir)


Processing directory: test-set\1-10
Questions file or rebuttal file does not exist, skippingtest-set\1-10.
Processing directory: test-set\1025
Questions file or rebuttal file does not exist, skippingtest-set\1025.
Processing directory: test-set\gmd-18-3311-2025
Questions file or rebuttal file does not exist, skippingtest-set\gmd-18-3311-2025.
Processing directory: test-set\hgss-15-71-2024
Rebuttal split into 3 chunks.
Rebuttal split into 2 chunks.
Finding answer for question: What are the key issues related to distributed authorship in online encyclopedias, compared to the authorship of earlier encyclopedias?
No answer found in chunk, moving to next chunk...
No answer found in chunk, moving to next chunk...
Answer found in chunk: Online encyclopedias, like Wikipedia, have a distributed authorship, which is different from earlier encyclopedias where authors were known. This distributed authorship brings issues such as articles being poor or dominated by the erroneous views of non-expert

## Example

In [ ]:
# Send a simple prompt to the model
# model: select a model from the list of models obtained from client.list()
# messages: a list of messages containing the conversation history. Some models also 
# have a system message, to add this, make the first message:
# {"role": "system", "content": "Your system message here"}
# Gemma3 does not have a system message, so we can start with the user message.
# To add responses from the model, you can use the "assistant" role, i.e.:
# {"role": "assistant", "content": "The capital of France is Paris."}

response = client.chat(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": "What is the capital of France?"},
        {"role": "assistant", "content": "The capital of France is Paris."},
        {"role": "user", "content": "And who many people live there?"},
    ],
)
print(response.message.content)

<think>
Alright, let me break down what's happening here. The user previously asked about the capital of France and I told them it's Paris. Now they're following up with a question about how many people live there. 

Hmm, the user wrote "who many people" instead of "how many." That's a common typo, so I should make sure to correct that in my response without pointing out the mistake explicitly.

They’re probably looking for population statistics. Since Paris is a major city, they might be planning a trip, doing research, or just curious about its size relative to other cities. 

I remember that Paris has around 2.165 million people as of recent estimates. But it's also part of a larger metropolitan area called Île-de-France, which has over 12 million residents. That makes the metro area one of the largest in Europe.

I should provide both numbers because the user might be interested in either the city proper or the broader area. It gives them a clearer picture depending on their needs.

In [ ]:
# To control the decoding parameters, such as temperature, maxium number of tokens, etc.,
# you can pass additional parameters to the chat method.
# For a complete list of options, check the Ollama API documentation at:
# https://github.com/ollama/ollama/blob/main/docs/modelfile.md#valid-parameters-and-values
x = [
    "How does the unit of wave power translate to W/m (Watts per meter) from the provided formula, which is the product of squared significant height and wave period?",
    "What is the reference source for the wave power formula used in the study?",
    "How can wave power be negative, given the observed range of -2000 to 2000 W/m in Figure 2?",
    "Why does the latitude scale on the y-axis in Figures 2b, c, d, and e not maintain a fixed distance, as it does in Figure 2a?",
    "What is the rationale behind the chosen color palette, where higher wave power and sea level are represented in reddish tones and lower values in bluish tones, while this representation is reversed for wave direction and waterline position?",
    "What was the specific reason for removing 40% of transects from non-sandy beaches?",
    "What is the unit of measurement for wave energy in Figure S3?",
    "Why is the wave energy formula in Figure S3 the same as the wave power formula in Figure 2?",
    "What was the basis for dividing the North American West Coast (NAWC) into five subregions?",
    "What is the rationale for using a rectangular boundary in Figure 1?",
    "Were any additional studies conducted to determine if parameters such as beach slope, substrate lithology,or riverine sediment inputs significantly affect waterline positions on a seasonal scale?"]
    
response = client.chat(
    model=MODEL_NAME,
    messages=[
        {"role": "user", 
         "content": "From the following list of questions, extract only information seeking question and question with no reference to figrues, line etc : "
         "How does the unit of wave power translate to W/m (Watts per meter) from the provided formula, which is the product of squared significant height and wave period?"
         "What is the reference source for the wave power formula used in the study?"
         "How can wave power be negative, given the observed range of -2000 to 2000 W/m in Figure 2?"
         "Why does the latitude scale on the y-axis in Figures 2b, c, d, and e not maintain a fixed distance, as it does in Figure 2a?"
         },
    ],
    options={
        "temperature": 0.5,
        "num_predict": 1024,
    }
)
print(response.message.content)

Here are the information-seeking questions from the list, excluding those referencing figures/lines/specific locations:

*   **What is the reference source for the wave power formula used in the study?**
*   **How does the unit of wave power translate to W/m (Watts per meter) from the provided formula, which is the product of squared significant height and wave period?**



The other two questions specifically ask about elements *within* figures (Figure 2, Figures 2b,c,d,e) and are therefore excluded based on your criteria.


In [ ]:
# To force the model to generate a structured response, i.e. a JSON object,
# you can define a schema for the response and pass it. The schema can be defined using 
# Pydantic models and the built-in python types (more complex types are also supported, 
# check the pydantic documentation for more details).

from pydantic import BaseModel
class PopulationResponse(BaseModel):
    city: str
    population: int

response = client.chat(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": "What is the capital of France?"},
        {"role": "assistant", "content": "The capital of France is Paris."},
        {"role": "user", "content": "And who many people live there?"},
    ],
    format=PopulationResponse.model_json_schema()
)
structured_response = PopulationResponse.model_validate_json(response.message.content)
print(structured_response)

city='Paris' population=2148000


In [ ]:
prompt = (
    "Identify and extract all **scientific, information-seeking questions** posed by the reviewer.\n\n"
    "Each question must:\n"
    "- Seek clarification or justification about methods, data, assumptions, results, or interpretations\n"
    "- Be answerable by the authors with scientific explanation\n"
    "Do NOT include:\n"
    "- Editorial question about grammar, style, or formatting\n"
    "- Rhetorical comment\n"
    "- Question regarding figures\n"
    
    "Rephrase the question to be self-contained and precise. Remove reference to any line or table number. If context loses after removal, discard the question.\n\n"

    "### Examples:\n"
    "Input: 'In Table 2, the authors claim the signal-to-noise ratio improved. Was it over multiple trials or one run?'\n"
    "Output: [\"How was the signal-to-noise ratio measured in the modified setup—over multiple trials or a single run?\"]\n\n"

    "Input: 'The manuscript overinterprets the data.'\n"
    "Output: []\n\n"

    "Now extract the questions from this review:\n\n{REVIEW}"
)

In [ ]:
import openai
from openai import OpenAI
from pydantic import BaseModel

openai.api_key =

review_text = "I am now satisfied with the changes that have been made by the authors, and these revisions have made an improvement from the first version of the article. \nThere are a few other minor concerns that the author may want to be consider; 1. All the data presented in this manuscript are generated by T cell lines or T cell \u201cclone\u201d. D10 cells are T lymphoblasts that continuously proliferate without stimulation, thereby exhibiting cancer cell properties. It is important to demonstrate this phenomenon occuring in vivo using primary T cells. \n2. It isn\u2019t very unclear how TCR internalization was measured by using anti-TCR antibody in the beginning and the end of incubation; this could be explained better. \n3. In the abstract, it would read better if the word \u201cinterestingly\u201d was deleted from the start of the second paragraph. "


client = OpenAI(api_key=openai.api_key)

class ScientificQuestions(BaseModel):
    questions: list[str]

response = client.responses.parse(
    model="o3-2025-04-16",
    input=[
        {"role": "system", "content": "Extract scientific questions from given peer reviews and return them as a structured list."},
        {"role": "user", "content": prompt.replace("{REVIEW}", review_text)}
    ],
    text_format=ScientificQuestions
)

# Access the parsed output
questions = response.output_parsed
print(questions)

NotFoundError: Error code: 404 - {'error': {'message': 'Your organization must be verified to use the model `o3-2025-04-16`. Please go to: https://platform.openai.com/settings/organization/general and click on Verify Organization. If you just verified, it can take up to 15 minutes for access to propagate.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}